In [7]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Jan 26 03:00:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/

In [8]:
!nvcc --version

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [10]:
import torch 
torch.__version__

'1.13.1+cu117'

# unoptimized onnx

In [1]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM


In [2]:

tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")
model_ort = ORTModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2", from_transformers=False, provider='CUDAExecutionProvider')


2023-01-26 03:02:11.167997716 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-26 03:02:11.168031130 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-26 03:02:15.520062534 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-26 03:02:15.520093280 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-26 03:02:19.393265478 [W:onnxrun

In [3]:
model_ort.device

device(type='cuda', index=0)

In [4]:
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')

In [5]:
%%timeit 
outputs = model_ort.generate(**inputs, num_beams=3, num_return_sequences=3)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


181 ms ± 6.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
outputs = model_ort.generate(**inputs)

134 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# compare with optimized onnx (no real difference, optimized onnx is actually slower)

In [7]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")
model_ort = ORTModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2_optimized", from_transformers=False, provider='CUDAExecutionProvider')


2023-01-26 03:02:45.213768679 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-26 03:02:45.213798293 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-26 03:03:00.441794726 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-26 03:03:00.441829182 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-26 03:03:15.654116970 [W:onnxrun

In [8]:
model_ort.device

device(type='cuda', index=0)

In [9]:
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')

In [10]:
%%timeit 
outputs = model_ort.generate(**inputs, num_beams=3, num_return_sequences=3)

183 ms ± 978 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit
outputs = model_ort.generate(**inputs)

138 ms ± 1.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# compare with pytorch

In [12]:
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoModel,
    AutoTokenizer
)

In [13]:
model_pt = AutoModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2")

In [14]:
tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")

In [15]:
model_pt.eval()
model_pt.cuda()
tmp = 2

In [16]:
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt").to('cuda')

## vanilla - ONNX is (333-181)/333 = 45% faster for beam-search, (241-134) / 241 = 44% faster for greedy-decoding

In [17]:
%%timeit
outputs = model_pt.generate(**inputs, num_beams=3, num_return_sequences=3)

333 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
outputs = model_pt.generate(**inputs)

241 ms ± 4.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## inference mode - ONNX is (306-181)/306 = 41% faster for beam-search, (219-134)/219 = 39% faster for greedy-decoding

In [20]:
import torch

In [21]:
%%timeit
with torch.inference_mode():
    outputs = model_pt.generate(**inputs, num_beams=3, num_return_sequences=3)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


306 ms ± 2.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
with torch.inference_mode():
    outputs = model_pt.generate(**inputs)

219 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## fp16 autocast inference mode (actually slower than without fp16)

In [23]:
%%timeit
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    outputs = model_pt.generate(**inputs, num_beams=3, num_return_sequences=3)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


314 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    outputs = model_pt.generate(**inputs)

238 ms ± 3.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
